In [1]:
import pandas as pd
import numpy as np
import h2o
from h2o.automl import H2OAutoML

ModuleNotFoundError: No module named 'h2o'

In [ ]:
# Load data
train_data = pd.read_csv('../data/train_AIC.csv')

In [ ]:
# Normalization
constmaxes = pd.DataFrame(train_data.describe().T['max']).T
prepared = {}
for i in train_data.columns[:-1]:
    prepared[i] = np.array(train_data[i]) / constmaxes[i][0]
prepared = pd.DataFrame(pd.DataFrame(prepared).T)
prepared_X = prepared.values

prepared_y = train_data['y'].values

In [ ]:
# Initialize H2O
h2o.init()

In [ ]:
# Convert data to H2O Frame
h2o_df = h2o.H2OFrame(prepared)
h2o_df['y'] = h2o.H2OFrame(prepared_y)

In [ ]:
# Define AutoML
aml = H2OAutoML(max_models=10, seed=42)

In [ ]:
# Train AutoML
aml.train(y='y', training_frame=h2o_df)

In [ ]:
# Get best model
best_model = aml.leader

In [ ]:
# Cross-validation using AutoML's built-in function
cv_results = best_model.cross_validation()

In [ ]:
# Print CV results
print(cv_results)

In [ ]:
# Train the best model on the full dataset
best_model.train(y='y', training_frame=h2o_df)

In [ ]:
# Shutdown H2O
h2o.cluster().shutdown()